In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

*The content in this section is adapted from Andrew Ng's excellent
Coursera course, available here:* https://www.coursera.org/course/ml

The issues associated with validation and 
cross-validation are some of the most important
aspects of the practice of machine learning.  Selecting the optimal model
for your data is vital, and is a piece of the problem that is not often
appreciated by machine learning practitioners.

Of core importance is the following question:

**If our estimator is underperforming, how should we move forward?**

- Use simpler or more complicated model?
- Add more features to each observed data point?
- Add more training samples?

The answer is often counter-intuitive.  In particular, **sometimes using a
more complicated model will give _worse_ results.**  Also, **sometimes adding
training data will not improve your results.**  The ability to determine
what steps will improve your model is what separates the successful machine
learning practitioners from the unsuccessful.

## Learning Curves and Validation Curves

One way to address this issue is to use what are often called **Learning Curves**.
Given a particular dataset and a model we'd like to fit (e.g. using k neighbors regression), we'd
like to tune our value of the *hyperparameter* ``n_neighbors`` to give us the best fit.

Lets go back to our regression problem from earlier:

In [ ]:
from figures import plot_kneighbors_regularization
plot_kneighbors_regularization()

Learning Curves
============

What the right model for a dataset is depends critically on how much data we have. More data allows us to be more confident about building a complex model. Lets built some intuition on why that is. Look at the folling datasets:


In [ ]:
from figures import plot_regression_datasets
plot_regression_datasets()

They all come from the same underlying process. But if you were asked to make a prediction, you would be more likely to draw a straight line for the left-most one, as there are only very few datapoints, and no real rule is aparent. For the dataset in the middle, some structure is recognizable, though the exact shape of the true function is maybe not obvious. With even more data on the right hand side, you would probably be very comfortable with drawing a sinusoidal line with a lot of certainty.

A great way to explore how a model fit evolves with different dataset sizes are learning curves.
A learning curve plots the validation error for a given model against different training set sizes.

But first, take a moment to think about what we're going to see:

**Questions:**

- **As the number of training samples are increased, what do you expect to see for the training error?  For the validation error?**
- **Would you expect the training error to be higher or lower than the validation error?  Would you ever expect this to change?**

We can run the following code to plot the learning curve for a ``d=1`` model:

# FIXME WORST EXAMPLE EVER

In [ ]:
from figures import make_dataset
from sklearn.learning_curve import learning_curve
from sklearn.neighbors import KNeighborsRegressor

x, y = make_dataset(n_samples=100)
X = x[:, np.newaxis]

training_sizes, train_scores, test_scores = learning_curve(KNeighborsRegressor(n_neighbors=5), X, y, cv=10)
plt.plot(training_sizes, train_scores.mean(axis=1), label="training scores")
plt.plot(training_sizes, test_scores.mean(axis=1), label="test scores")
plt.legend(loc='best')

You can see that for the very complex model with ``n_neighbors=2``, the score increase 

Notice that the validation error *generally decreases* with a growing training set,
while the training error *generally increases* with a growing training set.  From
this we can infer that as the training size increases, they will converge to a single
value.

From the above discussion, we know that `d = 1` is a high-bias estimator which
under-fits the data. This is indicated by the fact that both the
training and validation errors are very high. When confronted with this type of learning curve,
we can expect that adding more training data will not help matters: both
lines will converge to a relatively high error.

**When the learning curves have converged to a high error, we have an underfitting model.**

An underfitting model can be improved by:

- Using a more sophisticated model (i.e. in this case, increase ``d``)
- Gather more features for each sample.
- Decrease regularlization in a regularized model.

A underfitting model cannot be improved, however, by increasing the number of training
samples (do you see why?)

Now let's look at an over-fit model:

# FIXME ADD EXAMPLE HERE

Here we show the learning curve for `d = 20`. From the above
discussion, we know that `d = 20` is an estimator
which **over-fits** the data. This is indicated by the fact that the
training error is much less than the validation error. As
we add more samples to this training set, the training error will
continue to climb, while the cross-validation error will continue
to decrease, until they meet in the middle. In this case, our
intrinsic error was set to 1.0, and we can infer that adding more
data will allow the estimator to very closely match the best
possible cross-validation error.

**When the learning curves have not yet converged with our full training set, it indicates an over-fit model.**

An overfitting model can be improved by:

- Gathering more training samples.
- Using a less-sophisticated model (i.e. in this case, make ``d`` smaller)
- Increasing regularization.

In particular, gathering more features for each sample will not help the results.

Validation Curves
===========
FIXME REWRITE THIS SECTION!

In order to quantify the effects of bias and variance and construct
the best possible estimator, we will split our training data into
a *training set* and a *validation set*.  As a general rule, the
training set should be about 60% of the samples.

The overarching idea is as follows. The model parameters (in our case,
the coefficients of the polynomials) are learned using the training
set as above. The error is evaluated on the validation set,
and the meta-parameters (in our case, the degree of the polynomial)
are adjusted so that this validation error is minimized.
Finally, the labels are predicted for the test set. These labels
are used to evaluate how well the algorithm can be expected to
perform on unlabeled data.

The validation error of our polynomial regression can be visualized
by plotting the error as a function of the polynomial degree d. We can do
this as follows:

In [ ]:
# suppress warnings from Polyfit
import warnings
warnings.filterwarnings('ignore', message='Polyfit*')

degrees = np.arange(21)
train_err = np.zeros(len(degrees))
validation_err = np.zeros(len(degrees))

for i, d in enumerate(degrees):
    p = np.polyfit(xtrain, ytrain, d)

    train_err[i] = compute_error(xtrain, ytrain, p)
    validation_err[i] = compute_error(xtest, ytest, p)

fig, ax = plt.subplots()

ax.plot(degrees, validation_err, lw=2, label = 'cross-validation error')
ax.plot(degrees, train_err, lw=2, label = 'training error')
ax.plot([0, 20], [error, error], '--k', label='intrinsic error')

ax.legend(loc=0)
ax.set_xlabel('degree of fit')
ax.set_ylabel('rms error')

This figure compactly shows the reason that validation is
important. On the left side of the plot, we have very low-degree
polynomial, which under-fits the data. This leads to a very high
error for both the training set and the validation set. On
the far right side of the plot, we have a very high degree
polynomial, which over-fits the data. This can be seen in the fact
that the training error is very low, while the validation
error is very high. Plotted for comparison is the intrinsic error
(this is the scatter artificially added to the data: click on the
above image to see the source code). For this toy dataset,
error = 1.0 is the best we can hope to attain. Choosing `d=6` in
this case gets us very close to the optimal error.

The astute reader will realize that something is amiss here: in
the above plot, `d = 6` gives the best results. But in the previous
plot, we found that `d = 6` vastly over-fits the data. What’s going
on here? The difference is the **number of training points** used.
In the previous example, there were only eight training points.
In this example, we have 100. As a general rule of thumb, the more
training points used, the more complicated model can be used.
But how can you determine for a given model whether more training
points will be helpful? A useful diagnostic for this are learning curves.

## Summary

We’ve seen above that an under-performing algorithm can be due
to two possible situations: under-fitting and over-fitting. In order to evaluate our algorithm, we
set aside a portion of our training data for cross-validation.
Using the technique of learning curves, we can train on progressively
larger subsets of the data, evaluating the training error and
cross-validation error to determine whether our algorithm is overfitting or underfitting. But what do we do with this information?

### Underfitting

If our algorithm is **underfitting**, the following actions might help:

- **Add more features**. In our example of predicting home prices,
  it may be helpful to make use of information such as the neighborhood
  the house is in, the year the house was built, the size of the lot, etc.
  Adding these features to the training and test sets can improve
  the fit.
- **Use a more sophisticated model**. Adding complexity to the model can
  help improve the fit. For a polynomial fit, this can be accomplished
  by increasing the degree d. Each learning technique has its own
  methods of adding complexity.
- **Use fewer samples**. Though this will not improve the classification,
  an underfitting algorithm can attain nearly the same error with a smaller
  training sample. For algorithms which are computationally expensive,
  reducing the training sample size can lead to very large improvements
  in speed.
- **Decrease regularization**. Regularization is a technique used to impose
  simplicity in some machine learning models, by adding a penalty term that
  depends on the characteristics of the parameters. If a model is underfitting,
  decreasing the regularization can lead to better results.
  
### Overfitting

If our algorithm shows signs of **overfitting**, the following actions might help:

- **Use fewer features**. Using a feature selection technique may be
  useful, and decrease the over-fitting of the estimator.
- **Use a simpler model**.  Model complexity and over-fitting go hand-in-hand.
- **Use more training samples**. Adding training samples can reduce
  the effect of over-fitting.
- **Increase Regularization**. Regularization is designed to prevent
  over-fitting. So increasing regularization
  can lead to better results for overfitting models.

These choices become very important in real-world situations. For example,
due to limited telescope time, astronomers must seek a balance between
observing a large number of objects, and observing a large number of
features for each object. Determining which is more important for a
particular learning task can inform the observing strategy that the
astronomer employs. In a later exercise, we will explore the use of
learning curves for the photometric redshift problem.